In [1]:
import pandas as pd

import bs4 as bs4

import json
import glob
import tqdm

In [2]:
with open("./data_extracted.json", "w+") as output:
    
    for video in tqdm.tqdm_notebook(glob.glob("skoob/video_*.html")):

        print(video)
        
        with open (video, "r+") as inp:
            page_html = inp.read()
            parsed_page = bs4.BeautifulSoup(page_html)
            
            data = {}

            # Adicionando o link do video nos dados

            data["link"] = "/"+video[12:]
            
            # BOX_1: Extraindo "ANO" e "PAGINAS"             
            try:
                box_1 = parsed_page.find_all("div", {"class": "sidebar-desc"})

                b1 = box_1[0].get_text()

                index_ano = b1.find("Ano")
                index_pag = b1.find("Páginas")

                data["ano"] = information[index_ano+5:index_ano+9]
                data["pag"] = information[index_pag+9:index_pag+13]

            except:
                pass

            # BOX_2: Extraindo "RATINGS" (chamado de "avaliações" no site)
            try:
                box_2 = parsed_page.find_all("div", {"id": "livro-perfil-status"})

                b2 = box_2[0]
                data["ratings"] = b2.find("span").get_text()

            except:
                pass

            # BOX_3: Extraindo "RESENHAS", "ABANDONOS", "RELENDO", "QUEREM_LER", "LENDO", "LERAM"
            try:
                itens = ["resenhas", "abandonos", "relendo", "querem ler", "lendo", "leram"]

                box_3 = parsed_page.find_all("a", {"class": "text_blue"})

                for i in range(len(itens)):
                    data[itens[i]] = int(box_3[i].get_text().replace(".",""))

            except:
                pass
        
            # BOX_4: Extraindo "DESEJAM"
            try:
                box_4 = parsed_page.find_all("div", {"class": "span4 pg-livro-box-leitores-desejam"})

                b4 = box_4[0]

                data["desejam"] = int(b4.find("span", {"class": "box_titulo_total"}).get_text().replace(".",""))

            except:
                pass
        
            # BOX_5: Extraindo "MALE_READING" e "FEMALE_READING"
            try:
                box_5 = parsed_page.find_all("div", {"class": "span3 pg-livro-box-sexos"})

                b5 = box_5[0]

                data["male_reading"] = b5.find("span", {"class": "pg-livro-icone-male-label"}).get_text()

                data["female_reading"] = b5.find("span", {"class": "pg-livro-icone-female-label"}).get_text()

            except:
                pass

            output.write("{}\n".format(json.dumps(data)))

skoob/video_a-ultima-festa-1022977ed1117174.html
skoob/video_territorio-lovecraft-1050700ed1053247.html
skoob/video_os-guardioes-de-sangue-668853ed670661.html
skoob/video_floresta-encantada-440251ed498839.html
skoob/video_as-aventuras-do-tempo-1007147ed1009017.html
skoob/video_130-anos-em-busca-da-republica-880603ed886782.html
skoob/video_as-8-leis-espirituais-da-saude-635704ed637227.html
skoob/video_a-aventura-do-estilo-666582ed668282.html
skoob/video_o-garoto-da-loteria-671159ed673056.html
skoob/video_vender-e-humano-351275ed1012053.html
skoob/video_a-vista-infinita-11622118ed11652702.html
skoob/video_ferramentas-dos-titas-750662ed753887.html
skoob/video_shakti-o-poder-da-sua-essencia-11645873ed11663257.html
skoob/video_a-batalha-por-wondla-635887ed637416.html
skoob/video_a-arte-de-respirar-770209ed774388.html
skoob/video_amor-e-gelato-683238ed685660.html
skoob/video_pre-suasao-683616ed686058.html
skoob/video_serpentario-923534ed930082.html
skoob/video_o-que-aconteceu-com-annie-85175

## Avaliando os dados extraídos

In [3]:
df = pd.read_json("./data_extracted.json", lines=True)

In [4]:
df

,link,ratings,resenhas,abandonos,relendo,querem ler,lendo,leram,desejam,male_reading,female_reading
0,/a-ultima-festa-1022977ed1117174.html,3.8,1054,87,2,8516,517,5333,1378.0,13%,87%
1,/territorio-lovecraft-1050700ed1053247.html,3.7,211,32,0,1540,250,889,252.0,35%,65%
2,/os-guardioes-de-sangue-668853ed670661.html,4.0,2,0,0,204,0,15,63.0,29%,71%
3,/floresta-encantada-440251ed498839.html,4.4,14,63,9,1414,381,956,238.0,10%,90%
4,/as-aventuras-do-tempo-1007147ed1009017.html,4.4,1,1,0,63,1,3,12.0,11%,89%
...,...,...,...,...,...,...,...,...,...,...,...
1195,/a-economia-da-paixao-11625748ed11656281.html,0.0,0,0,0,1,0,0,1.0,80%,20%
1196,/a-alma-indomavel-785295ed790285.html,4.1,3,0,0,664,16,37,38.0,13%,87%
1197,/fdeu-geral-861723ed1170812.html,3.7,123,80,6,3367,359,1295,584.0,22%,78%
1198,/voce-748991ed752128.html,3.7,220,160,2,2810,226,2363,861.0,12%,88%
